# Susceptibility and infectiousness in mixing matrices
From the previous two notebooks,
we hopefully have some idea of what the mixing matrices 
we can use in our models represent.
This is really important basic knowledge before we start trying to
manipulate them.
This notebook aims to really nail down the intuition 
around the rows and columns in these matrices.
This notebook is again not that impressive visually,
but should continue to build our intuition around mixing matrices.

Let's think about adapting our mixing matrix,
starting off with a simple density-dependent transmission model
stratified into two arbitrary categories
(similar to one of the models from the previous notebook).
To get started, let's define a simple framework for a model
that we can add more features to later.
Also, let's define a simple two-stratum stratification object
that will expect a mixing matrix later on.

In [ ]:
try:
    import google.colab
    %pip install summerepi2
except:
    pass

In [ ]:
import pandas as pd
import numpy as np
pd.options.plotting.backend = "plotly"
from jax import numpy as jnp

from summer2 import CompartmentalModel, Stratification
from summer2.parameters import Parameter, Function

In [ ]:
def build_sir_model(
    config: dict,
) -> CompartmentalModel:
    
    compartments = config["compartments"]
    analysis_times = (0.0, config["end_time"])
    model = CompartmentalModel(
        times=analysis_times,
        compartments=compartments,
        infectious_compartments=("infectious",),
    )
    model.set_initial_population(
        distribution=
        {
            "susceptible": config["population"] - config["seed"], 
            "infectious": config["seed"],
        },
    )
    
    model.add_infection_density_flow(
        name="infection", 
        contact_rate=Parameter("risk_per_contact"),
        source="susceptible", 
        dest="infectious",
    )
    model.add_transition_flow(
        name="recovery", 
        fractional_rate=1.0 / Parameter("infectious_period"),
        source="infectious", 
        dest="recovered",
    )
    
    model.request_output_for_compartments(
        "prevalence",
        "infectious",
    )
    
    return model

In [ ]:
def build_simple_strat(
    compartments: list,
    mixing_matrix: jnp.array,
) -> Stratification:
                
    mix_strat = Stratification(
        "groups",
        ["group1", "group2"],
        compartments,
    )
    
    prop1 = Parameter("prop1")
    prop2 = 1.0 - prop1
    mix_strat.set_population_split(
        {
            "group1": prop1,
            "group2": prop2,
        }
    )

    if mixing_matrix is not None:
        mix_strat.set_mixing_matrix(mixing_matrix)

    return mix_strat

In [ ]:
model_config = {
    "end_time": 40.0,
    "population": 1.0,
    "seed": 0.01,
    "compartments": (
        "susceptible", 
        "infectious", 
        "recovered",
    ),
}
parameters = {
    "risk_per_contact": 0.5,
    "infectious_period": 4.0,
    "prop1": np.random.uniform(),
    "transmission_scaler": 2.0,
}

### Comparison model
For later comparison,
let's quickly build and run an unstratified model with the base parameters.

In [ ]:
base_model = build_sir_model(model_config)
base_model.run(parameters=parameters)

## Increased susceptibility for a sub-population

### Adjusting the infection rate
Next, let's increase the susceptibility of a stratum of the model,
by adjusting the rate of infection for one of our model sub-groups
(using `summer`'s adjustments structures).
This scales the rate of rate of infection for `group1` to be
the product of the `risk_per_contact` and the `susceptibility` parameter.
Increasing the effective parameter for the infection process
for a population stratum means that they will experience a greater
force of infection and so can be thought of as being more susceptible.
Note we haven't implemented a mixing matrix for this model.

In [ ]:
suscept_model = build_sir_model(model_config)
suscept_adjust_strat = build_simple_strat(model_config["compartments"], None)
suscept_adjust_strat.set_flow_adjustments(
    "infection",
    {
        "group1": Parameter("transmission_scaler"),  # Increased susceptibility for group1
        "group2": None,  # No change for group2
    },
)
suscept_model.stratify_with(suscept_adjust_strat)
suscept_model.run(parameters=parameters)

### Adjusting the mixing matrix
Next, let's create a model with a mixing matrix,
but with the first row
(relating to the infection of the `group1` population)
multiplied through by a value to represent increased susceptibility for this stratum.
The `susceptibility` parameter again increases the force of infection
for the `group1` population, by multiplying both of the contributions
to the force of infection through by the same value.

Remember from [notebook 12](./12-heterogeneous-mixing-intro.ipynb) 
that the force of infection for `group1`
is calculated as the number of infectious persons in `group1`
multiplied by the top-left cell of our mixing matrix,
plus the number of infectious persons in `group2`
multiplied by the top-right cell of our mixing matrix.
More generally, the force of infection is calculated
from the vector of the number of infectious persons in each sub-group
multiplied by the row of the matrix that pertains to the
population whose force of infection we are calculating.

In [ ]:
suscept_matrix_model = build_sir_model(model_config)

def build_suscept_adjusted_matrix(scaler):
    return jnp.array(
        [
            [scaler, scaler],  # Multiply group1 row of matrix by the susceptibility value
            [1.0, 1.0],
        ]
    )

mixing_matrix = Function(
    build_suscept_adjusted_matrix, 
    (Parameter("transmission_scaler"),),
)
suscept_matrix_strat = build_simple_strat(model_config["compartments"], mixing_matrix)
suscept_matrix_model.stratify_with(suscept_matrix_strat)
suscept_matrix_model.run(parameters=parameters)

### Adjusting the infection rate, with dummy mixing matrix
One last way to do the same thing
is that we could keep the unadjusted mixing matrix in place,
but still apply the adjustment to the infection process.
This might be a clearer illustration of how the adjustment
affects the model relative to the base heterogeneous mixing model,
although it's really very similar to the first approach, of course.

In [ ]:
suscept_matrix_adjust_model = build_sir_model(model_config)
mixing_matrix = jnp.array(
    [
        [1.0, 1.0],
        [1.0, 1.0],
    ]
)
suscept_matrix_adjust_strat = build_simple_strat(model_config["compartments"], mixing_matrix)
suscept_matrix_adjust_strat.set_flow_adjustments(
    "infection",
    {
        "group1": Parameter("transmission_scaler"),
        "group2": None,
    },
)
suscept_matrix_adjust_model.stratify_with(suscept_matrix_adjust_strat)
suscept_matrix_adjust_model.run(parameters=parameters)

### Checking the results
Let's check that these three different approaches to 
increasing susceptibility have the same behaviour 
(and also compare to the base model
with the baseline level of susceptibility).
Again, toggle the plotly curves on and off to check 
that they overly one another.

In [ ]:
outputs = pd.DataFrame(
    {
        "increased infection rate (stratified)": suscept_model.get_derived_outputs_df()["prevalence"],
        "adjusted matrix (stratified)": suscept_matrix_adjust_model.get_derived_outputs_df()["prevalence"],
        "increased infection rate, dummy matrix (stratified)": suscept_matrix_model.get_derived_outputs_df()["prevalence"],
    }
)
differences = outputs.min(axis=1) - outputs.max(axis=1)
assert all(abs(differences) < 1e-8), "There's a discrepancy"
outputs["base comparison (unstratified)"] = base_model.get_derived_outputs_df()["prevalence"]
outputs.plot()

## Adjusting infectiousness
Next, let's work through a similar process for infectiousness,
demonstrating that this is equivalent to changing
a column of the mixing matrix.

### Adjusting infectiousness
Again, we'll use `summer`'s adjustment methods here,
but now adjusting the contribution of the infectious person to the force of infection,
rather than adjusting the rate of infection itself.

In [ ]:
infectious_model = build_sir_model(model_config)
infectious_adjust_strat = build_simple_strat(model_config["compartments"], None)
infectious_adjust_strat.add_infectiousness_adjustments(
    "infectious",
    {
        "group1": Parameter("transmission_scaler"),
        "group2": None,
    }
)
infectious_model.stratify_with(infectious_adjust_strat)
infectious_model.run(parameters=parameters)

### Adjusting the mixing matrix
This is similar to the adjustment for susceptibility,
except that we'll scale a column of the mixing matrix rather than row.

Here, we're increasing the extent to which one of the population
groups contributes to each of the force of infection calculations,
so increasing infectiousness.

In [ ]:
infectious_matrix_model = build_sir_model(model_config)

def build_infectious_adjusted_matrix(scaler):
    return jnp.array(
        [
            [scaler, 1.0],  # Multiply group1 column of the matrix by the susceptibility value
            [scaler, 1.0],  # Same again
        ]
    )

mixing_matrix = Function(
    build_infectious_adjusted_matrix, 
    (Parameter("transmission_scaler"),),
)
infectious_matrix_strat = build_simple_strat(model_config["compartments"], mixing_matrix)
infectious_matrix_model.stratify_with(infectious_matrix_strat)
infectious_matrix_model.run(parameters=parameters)

### Adjusting infectiousness, with dummy mixing matrix
As before, this is a bit unnecessary,
but might make the comparison more obvious,
because the model for which we are adjusting the
infectiousness for is the stratified model.

In [ ]:
infectious_matrix_adjust_model = build_sir_model(model_config)
mixing_matrix = jnp.array(
    [
        [1.0, 1.0],
        [1.0, 1.0],
    ]
)
infectious_matrix_adjust_strat = build_simple_strat(model_config["compartments"], mixing_matrix)
infectious_matrix_adjust_strat.add_infectiousness_adjustments(
    "infectious",
    {
        "group1": Parameter("transmission_scaler"),
        "group2": None,
    }
)
infectious_matrix_adjust_model.stratify_with(infectious_matrix_adjust_strat)
infectious_matrix_adjust_model.run(parameters=parameters)

### Checking the results
As we did for susceptibility, 
let's check that these three different approaches to 
increasing infectiousness behave the same
(and also compare to the base model
with the baseline level of infectiousness).

In [ ]:
outputs = pd.DataFrame(
    {
        "increased infectiousness (stratified)": infectious_model.get_derived_outputs_df()["prevalence"],
        "adjusted matrix (stratified)": infectious_matrix_model.get_derived_outputs_df()["prevalence"],
        "increased infectiousness, dummy matrix (stratified)": infectious_matrix_adjust_model.get_derived_outputs_df()["prevalence"],
    }
)
differences = outputs.min(axis=1) - outputs.max(axis=1)
assert all(abs(differences) < 1e-8), "There's a discrepancy"
outputs["base comparison"] = base_model.get_derived_outputs_df()["prevalence"]
outputs.plot()

## Summary
Last, also note that even though we can make each of these two adjustments
to our model in multiple ways,
increasing the infectiousness of a group and increasing the susceptibility of a group
result in different dynamics (even if the effect is fairly similar in this simplistic example)
and are very different processes conceptually.

Note that the point here was to build intuition for what these matrices mean.
Having done all of this, we're actually **_not_** generally going to go around multiplying
rows and columns of our matrices by scalar values to adjust the susceptibility
and infectiousness of population groups.
The reason for this is that we want to keep the mixing matrices for representing 
the rates at which population groups come into contact with one another,
and we have other ways to adjust susceptibility and infectiousness 
that we would prefer to use.
If we isolate these two processes and save the model's mixing matrix
for just the rates at which sub-populations interact,
we can preserve separate intuitions for what each of these structures
mean within our model.